In [1]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [2]:
!nvidia-smi

Thu Mar 17 11:39:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    73W / 250W |  14023MiB / 16280MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

Processing the data.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [4]:
# %cd $PATH_TO_FOLDER
%cd ../../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/AIEEEGenderData/Preprocessing/')
from aieee_gender import *

In [6]:
aieee_df = AIEEEGenderData()

In [7]:
%%capture
!pip install datasets

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


from transformers import AutoConfig
config = AutoConfig.from_pretrained("bert-base-uncased")

In [9]:
from sklearn.model_selection import train_test_split

X = list(aieee_df["Name"])
y = list(aieee_df["Gender"].values.astype('int'))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)

In [10]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [11]:
from transformers import TrainingArguments
from transformers import Trainer

In [12]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    f_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    f_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    m_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    m_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    m_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "female_precision": f_precision, "female_recall":f_recall, "female_f1": f_f1,
        "male_precision": m_precision, "male_recall": m_recall, "male_f1": m_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Step,Training Loss,Validation Loss,Accuracy,Female Precision,Female Recall,Female F1,Male Precision,Male Recall,Male F1
500,0.240700,0.174659,0.936426,0.910564,0.944067,0.927013,0.957027,0.930717,0.943689
1000,0.164700,0.139782,0.950801,0.946705,0.937744,0.942203,0.953811,0.960556,0.957172
1500,0.140100,0.125491,0.952780,0.958409,0.929936,0.943958,0.948787,0.969848,0.959202
2000,0.125400,0.115968,0.957487,0.956396,0.943610,0.949960,0.958283,0.967856,0.963046


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
metrics=trainer.evaluate()
print(metrics)

{'eval_loss': 0.10995952785015106, 'eval_accuracy': 0.9611193458537618, 'eval_female_precision': 0.9589992115232985, 'eval_female_recall': 0.9496848159363157, 'eval_female_f1': 0.9543192865617943, 'eval_male_precision': 0.9626767626824134, 'eval_male_recall': 0.9696629053602174, 'eval_male_f1': 0.9661572052401746, 'eval_runtime': 208.3682, 'eval_samples_per_second': 589.274, 'eval_steps_per_second': 5.893, 'epoch': 1.0}


In [15]:
# # Load test data
# test_data = pd.read_csv("test.csv")
# X_test = list(test_data["review"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [16]:
metrics

{'eval_loss': 0.10995952785015106,
 'eval_accuracy': 0.9611193458537618,
 'eval_female_precision': 0.9589992115232985,
 'eval_female_recall': 0.9496848159363157,
 'eval_female_f1': 0.9543192865617943,
 'eval_male_precision': 0.9626767626824134,
 'eval_male_recall': 0.9696629053602174,
 'eval_male_f1': 0.9661572052401746,
 'eval_runtime': 208.3682,
 'eval_samples_per_second': 589.274,
 'eval_steps_per_second': 5.893,
 'epoch': 1.0}

In [17]:
model.save_pretrained( "Models/AIEEEGenderData/BERT/SavedModel/BERT")
